# Message Queue: RabbitMQ Pika Subscriber
## Illustrates distributed computing using message queues.

We will use multiple implementations of message queues including <b>RabbitMQ/pika</b> and <b>MQTT</b>. This notebook is an illustration of subscribing or receiving a stream using RabbitMQ.
<br>
<br>
The publisher and subscriber communicate through a specified <i>routing key</i>, <i>exchange</i>, and <i>host</i>. In this example, the data consists of a list of lists, and each sublist of data is published separately after which the sender pauses.
<br>
<br>
<b>Note</b>: Run this notebook <i>ExamplePikaSubscriber</i> first. Then run the notebook: <i>ExamplePikaPublisher</i>. You should see the data that you published using the notebook <i>ExamplePikaPublisher</i> printed in this notebook <i>ExamplePikaSubscriber</i>.


# The Central Idea: Callback extends Stream

In [1]:
def callback(ch, method, properties, body):
    data=json.loads(body)
    if data[-1] == '_finished':
        extend_stream(procs, data[:-1], stream_name)
        terminate_stream(procs, stream_name)
        sys.exit()
    else:
        extend_stream(procs, data, stream_name)

# An Example
Next we give an example using threads; you can use the example if you want to subscribe to a stream in an application with multiple processes.

In [2]:
import pika
import json
import threading

import sys
sys.path.append("../")

from IoTPy.helper_functions.print_stream import print_stream

# multicore imports
from IoTPy.concurrency.multicore import get_processes, get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import get_proc_that_inputs_source
from IoTPy.concurrency.multicore import extend_stream
from IoTPy.concurrency.PikaSubscriber import PikaSubscriber

In [3]:
def pika_subscriber_test():
    """
    This function shows how to use Pika to receive a
    stream that was sent using PikaPublisher.

    The pika_receive_agent prints its single input stream.
    The pika_callback_thread receives JSON input (see 'body
    in callback) from Pika and puts the data into a stream
    called 'pika_receive_stream'.

    """
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.
    def pika_receive_agent(in_streams, out_streams):
        print_stream(in_streams[0], 'x')

    # Step 0.1: Define source thread targets (if any).
    def pika_callback_thread_target(procs):
        def callback(ch, method, properties, body):
            data=json.loads(body)
            if data[-1] == '_finished':
                # Received a '_finished' message indicating that the 
                # thread should terminate, and no further messages will
                # be arriving on this stream.
                # Extend the stream with data excluding'_finished'
                extend_stream(procs, data[:-1], stream_name='pika_receive_stream')
                terminate_stream(procs, stream_name='pika_receive_stream')
                # Terminate the callback thread.
                sys.exit()
            else:
                extend_stream(procs, data, stream_name='pika_receive_stream')
        # Declare the Pika subscriber
        pika_subscriber = PikaSubscriber(
            callback, routing_key='temperature',
            exchange='publications', host='localhost')
        pika_subscriber.start()

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs, additional arguments.
    # The pika_receive_stream data type is 'x' for arbitrary.
    multicore_specification = [
        # Streams
        [('pika_receive_stream', 'x')],
        # Processes
        [{'name': 'pika_receive_process', 'agent': pika_receive_agent,
          'inputs':['pika_receive_stream'], 'sources': ['pika_receive_stream']}]]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    pika_callback_thread = threading.Thread(
        target=pika_callback_thread_target, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # pika_callback_thread runs in the process called 'pika_receive_process'
    procs['pika_receive_process'].threads = [pika_callback_thread]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

In [4]:
pika_subscriber_test()

x[0] = 0
x[1] = 1
x[2] = Hello
x[3] = World
x[4] = THE
x[5] = END
x[6] = IS
x[7] = NIGH!
